In [ ]:
# Program places orders

# STATUS: Incomplete

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=1) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=1) # kavi IBG live

In [73]:
import pandas as pd
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
base_rom = 1.25    #90% RoM atleast

df_orders = pd.read_pickle(datapath+'df_orders.pkl')

# For puts place orders for strikes below threshold
df_orders = df_orders[df_orders.Strike <= df_orders.Threshold]

# Filter out the top Return on Margin trades
df_orders1 = df_orders[df_orders.RoM >= base_rom].reset_index(drop=True)

In [77]:
df_orders1.sort_values('ibSymbol').to_excel(datapath+'temp.xlsx')

#------------------------------------------------
#####    WARNING: THIS PLACES THE TRADE  #######
#________________________________________________

limitTrade = [ib.placeOrder(contract, order) 
              for contract, order in zip(df_orders1.Contract, df_orders1.Orders)]

#------------------------------------------------
#####         Code to cancel all trades    ############
#________________________________________________
cancel = ib.reqGlobalCancel()


In [80]:
# Put placed orders to a dataframe
df_orders_placed = pd.DataFrame(limitTrade)

In [81]:
# Write orders placed to file
x  =  datapath + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_orders_placed.xlsx"
writer = pd.ExcelWriter(x)
df_orders_placed.to_excel(writer, 'all-options', index=False, header=None)
writer.save()

In [82]:
ib.disconnect()